# **AI Workshop: From Agents to Intelligent Search**

## - Led by Ayaan Ahmad

### A hands-on guide on building and programming intelligent agents to solve complex problems.

# AI Workshop - Lab 2: Depth-First Search (DFS)

In this lab, we will implement our first search algorithm, DFS.
It explores as far as possible down one path before backtracking.

We will:
1. Load our standard maze from `workshop_maze.csv`.
2. Run the DFS algorithm.
3. Measure its execution time, the number of cells it explored, and the length of its final path.
4. Save these metrics to `performance_results.csv`.
5. Visualize both the search path and the final path.

## Install Libraries

In [ ]:
pip install Pillow

### Importing Libraries

In [3]:
from pyamaze import maze, agent, COLOR
import os
import glob
from PIL import Image
from queue import PriorityQueue
import time # To measure execution time
import pandas as pd # To handle CSV data
import matplotlib.pyplot as plt # For plotting
import seaborn as sns # For better visualizations


# Create a list to store performance results
performance_results = []

# Part 2: Uninformed Search (Finding a Path Blindly)

Our agent is in the maze, but it doesn't know how to get to the goal at (1,1). **Uninformed search algorithms** are methods that search a space without any information about the cost or distance to the goal. They just systematically explore until a solution is found.

### 2.1 - Depth-First Search (DFS)
DFS explores as far as possible down one branch before backtracking. It uses a LIFO (Last-In, First-Out) strategy, which is like a stack of plates. You put a new plate on top and you take the top one off first.

In [15]:
def DFS(m):
    start = (m.rows, m.cols)
    # The 'explored' list will track every cell visited in order
    explored, frontier, dfsPath = [start], [start], {} 
    
    while frontier:
        currCell = frontier.pop()
        # The goal check is removed to allow for full exploration
        if currCell == (1, 1): 
             break
                #ESNW -- Prioritizes WEST
        for d in "NWSE":
            if m.maze_map[currCell][d]:
                if d == 'E': childCell = (currCell[0], currCell[1] + 1)
                elif d == 'W': childCell = (currCell[0], currCell[1] - 1)
                elif d == 'N': childCell = (currCell[0] - 1, currCell[1])
                elif d == 'S': childCell = (currCell[0] + 1, currCell[1])
                if childCell in explored: 
                    continue
                explored.append(childCell) # Add to the main explored list
                frontier.append(childCell)
                dfsPath[childCell] = currCell
                
    # After full exploration, reconstruct the forward path
    fwdPath = {}
    cell = (1, 1)
    # Check if the goal was reachable
    if cell in dfsPath or cell == start:
        while cell != start:
            fwdPath[dfsPath[cell]] = cell
            cell = dfsPath[cell]
        
    # Return both the final path and the list of all explored cells
    return fwdPath, explored

In [18]:
# --- Run DFS ---
m = maze()
m.CreateMaze(loadMaze='workshop_maze.csv')

# --- Measure Execution Time ---
start_time = time.time()
final_path, explored_cells = DFS(m) 
end_time = time.time()
dfs_time = end_time - start_time
print(f"DFS completed in {dfs_time:.6f} seconds. Path length: {len(final_path)}. Explored cells: {len(explored_cells)}")

time_taken = end_time - start_time
explored_count = len(explored_cells)
path_length = len(final_path)

# Save Metrics to CSV
results_file = 'performance_results.csv'
new_data = {'Algorithm': 'DFS', 'Time (s)': time_taken, 'Explored Cells': explored_count, 'Path Length': path_length}
if os.path.exists(results_file):
    df = pd.read_csv(results_file)
    df = df[df.Algorithm != 'DFS']
    df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)
else:
    df = pd.DataFrame([new_data])
df.to_csv(results_file, index=False)
print(f"DFS performance data saved to '{results_file}'")

# --- 1. Demo the SEARCH path ---
explored_path = {explored_cells[i]: explored_cells[i+1] for i in range(len(explored_cells)-1)}
search_agent = agent(m, footprints=True, color=COLOR.cyan, filled=True, shape='square')
m.tracePath({search_agent: explored_path})

# --- 2. Demo the FINAL path ---
path_agent = agent(m, footprints=True, color=COLOR.yellow)
m.tracePath({path_agent: final_path})

DFS completed in 0.005999 seconds. Path length: 218. Explored cells: 493
DFS performance data saved to 'performance_results.csv'


In [6]:
m.run()

Observation: Notice how the DFS path is long and inefficient. It dives deep into the maze and takes many detours.